In [1]:
import time
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df = pd.concat([df,df_add,df_test])
df['vol_change'] = df.groupby(["ts_code"])['vol'].pct_change()
df['low_change'] = df.groupby(["ts_code"])['low'].pct_change()
df['high_change'] = df.groupby(["ts_code"])['high'].pct_change()
df['vol_change'] = df['vol_change'].fillna(0)
df['low_change'] = df['vol_change'].fillna(0)
df['high_change'] = df['vol_change'].fillna(0)
df =  df[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]
# l=0
# for arr in df['ts_code']:
#   df.loc[(df['ts_code'] ==arr) == True, '行业'] = l
#   l=l+1


FileNotFoundError: [Errno 2] File b'./TRAINSET_STOCK.csv' does not exist: b'./TRAINSET_STOCK.csv'

In [0]:
df['行业']=df['行业']-51

In [70]:
df = df[['pct_change','vol_change','high_change','low_change','y']]
df.head()

,pct_change,vol_change,high_change,low_change,y
0,1.33,0.000000,0.000000,0.000000,1
1,0.19,0.039698,0.039698,0.039698,1
2,-0.39,-0.134904,-0.134904,-0.134904,0
3,0.74,-0.087730,-0.087730,-0.087730,1
4,0.51,0.222145,0.222145,0.222145,1


# LSTM

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

# lstm


In [72]:
dataset = df.values
dataset

array([[ 1.33      ,  0.        ,  0.        ,  0.        ,  1.        ],
       [ 0.19      ,  0.03969754,  0.03969754,  0.03969754,  1.        ],
       [-0.39      , -0.13490358, -0.13490358, -0.13490358,  0.        ],
       ...,
       [-0.83      , -0.07570617, -0.07570617, -0.07570617,  0.        ],
       [-0.64      , -0.03678727, -0.03678727, -0.03678727,  0.        ],
       [-0.54      , -0.01516826, -0.01516826, -0.01516826,  0.        ]])

In [86]:
x = []
y = []
for i in range(len(dataset)-15):
    x.append(dataset[i:(i+10),:5].tolist())
    y.append(dataset[(i+10):(i+15),4,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(
    x, y, test_size=0.2, random_state=0)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape
# xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1]*xtrain.shape[2], 1))
# xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[1], 1))
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))
xtrain.shape


(32056, 10, 5)

In [75]:
kf = KFold(n_splits=5, shuffle=True, random_state=520)
for (train,test) in kf.split(xtrain):
     print(train.shape,test.shape)


(25644,) (6412,)
(25645,) (6411,)
(25645,) (6411,)
(25645,) (6411,)
(25645,) (6411,)


In [108]:
regressor = Sequential()
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (xtrain.shape[1], 5)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50,return_sequences = True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = 5))
regressor.add(Activation('sigmoid'))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(xtrain, ytrain, epochs = 100, batch_size = 50)

Epoch 1/100
32056/32056 [==============================] - 30s 930us/step - loss: 0.2478
Epoch 2/100
32056/32056 [==============================] - 23s 731us/step - loss: 0.2454
Epoch 3/100
32056/32056 [==============================] - 24s 734us/step - loss: 0.2430
Epoch 4/100
32056/32056 [==============================] - 24s 748us/step - loss: 0.2393
Epoch 5/100
32056/32056 [==============================] - 23s 726us/step - loss: 0.2348
Epoch 6/100
32056/32056 [==============================] - 23s 729us/step - loss: 0.2306
Epoch 7/100
32056/32056 [==============================] - 23s 728us/step - loss: 0.2273
Epoch 8/100
32056/32056 [==============================] - 24s 734us/step - loss: 0.2241
Epoch 9/100
32056/32056 [==============================] - 23s 728us/step - loss: 0.2207
Epoch 10/100
32056/32056 [==============================] - 23s 729us/step - loss: 0.2176
Epoch 11/100
32056/32056 [==============================] - 23s 732us/step - loss: 0.2143
Epoch 12/100
32056/

In [109]:
predicted_stock_price = regressor.predict(xtest)
a = []
for i in range(len(ytest)):
    fpr, tpr, thresholds = roc_curve(ytest[i], predicted_stock_price[i], pos_label=1)
    auc(fpr, tpr)
    a.append(auc(fpr, tpr))
a=pd.DataFrame(a)
a=a.dropna()
a.mean()

0    0.878099
dtype: float64

In [0]:
第一周预测结果对比

In [0]:
4.24号开始到结束后取五天 然后取十天作为输入

In [122]:
df_test1= pd.read_csv(r'./20190506_STOCK.csv')
df_test2 = pd.read_csv(r'./20190514_STOCK.csv')
df_test3 = pd.read_csv(r'./20190521_STOCK.csv')
df_test4 = pd.concat([df_test1,df_test2,df_test3])
# df_test4.iloc[1:3]
df_test4.head()
    


,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
0,801010,20190402,农林牧渔,3605.26,3485.11,3605.26,3513.06,-41.36,-1.16,236389.0,2761598.0,39.92,3.78,0
1,801020,20190402,采掘,2935.97,2905.01,2941.36,2935.37,22.18,0.76,282181.0,1732160.0,15.25,1.34,1
2,801030,20190402,化工,2971.25,2961.67,3011.61,2996.91,55.41,1.88,762659.0,7911222.0,18.94,2.06,1
3,801040,20190402,钢铁,2743.16,2711.59,2743.16,2732.47,0.80,0.03,195360.0,1062437.0,7.85,1.23,1
4,801050,20190402,有色金属,3195.75,3152.60,3261.40,3234.95,63.39,2.00,582889.0,5159532.0,32.45,2.26,1


In [0]:
range_index = (df_test4.trade_date > 20190422)
df_test4 = df_test4[range_index]
df_test4['vol_change'] = df_test4.groupby(["ts_code"])['vol'].pct_change()
df_test4['low_change'] = df_test4.groupby(["ts_code"])['low'].pct_change()
df_test4['high_change'] = df_test4.groupby(["ts_code"])['high'].pct_change()
df_test4 =  df_test4[['ts_code','trade_date','name','pct_change','vol_change','high_change','low_change','y']]
range_index = (df_test4.trade_date > 20190423)
df_test4 = df_test4[range_index]


In [144]:
df_me=pd.DataFrame(df_test4[['ts_code','name']])
df_me=df_me.drop_duplicates()
df_me
df8=df_me[['name']]
df8=df8.T
df8.values

array([['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']], dtype=object)

In [0]:

colums=['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']
# colums=[801010, 801020, 801030, 801040, 801050, 801080, 801110, 801120,
#         801130, 801140, 801150, 801160, 801170, 801180, 801200, 801210,
#         801230, 801250, 801260, 801270, 801280, 801300, 801710, 801720,
#         801730, 801740, 801750, 801760, 801770, 801780, 801790, 801880,
#         801890, 802600]
range_index = (df_test4.trade_date > 20190507)
df_final = df_test4[range_index]
df_pr=pd.DataFrame()
for arr in colums:
  df3 = df_final.loc[df_final['name']==arr]
  dataset2=df3.values
  x1=dataset2[0:10,3:8].tolist()
  x1=np.array(x1)
  x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
  ytest1=regressor.predict(x1)
  y_p=pd.DataFrame(ytest1)
  y_p=y_p.T
  y_p['name']=arr
  y_p['trade_date']=[20190522,20190523,20190524,20190527,20190528]
#   y_p[1,'trade_date']=20190523
#   y_p[2,'trade_date']=20190524
#   y_p[3,'trade_date']=20190525
#   y_p[4,'trade_date']=20190526
  y_p
  df_pr=pd.concat([df_pr,y_p])

df_pr=df_me.merge(df_pr,how='left', on='name')

df_pr.columns=['ts_code','name','p','trade_date']
df_pr=df_pr[['ts_code','trade_date','p']]
df_pr
df_pr.to_csv('./比赛2.csv')

In [85]:
y1 = []
y1_real1=df_test4.loc[df_test4['trade_date']==20190508]
y1_real2=df_test4.loc[df_test4['trade_date']==20190509]
y1_real3=df_test4.loc[df_test4['trade_date']==20190510]
y1_real4=df_test4.loc[df_test4['trade_date']==20190513]
y1_real5=df_test4.loc[df_test4['trade_date']==20190514]
for arr in df_test4['name']:
    df2 = df_test4.loc[df_test4['name']==arr]
    dataset1=df2.values
    x1=dataset1[0:10,3:8].tolist()
#     x1.append(dataset1[5:15,3:8].tolist())
#     x1.append(dataset1[10:20,3:8].tolist())
    y1.append(dataset1[10:15,7,np.newaxis].tolist())
#     y1.append(dataset1[15:20,7,np.newaxis].tolist())    
    x1=np.array(x1)
    ytest1=regressor.predict(x1)
#     y1.append(ytest1)
#     print(arr)
#     print(y1,ytest2)
# y1

ValueError: ignored

In [0]:
第二周预测结果对比

In [87]:
# i=0
# modle=[]
for train,test in kf.split(xtrain):
  a = []
  b= []
  X_train, X_valid= xtrain[train,:], xtrain[test,:]
  y_train, y_valid = ytrain[train,:], ytrain[test,:]
#   print(i, 'fold')
#   model=regressor.fit(X_train,y_train, epochs = 100, batch_size = 50)
#   predicted_stock_price1 =y_valid
#   for j in range(len(y_valid)):
#       fpr1, tpr1, thresholds1 = roc_curve(y_valid[j], y_valid[j], pos_label=1)
#       a.append(auc(fpr1, tpr1))
#   a=pd.DataFrame(a)
#   a=a.dropna()
#   print(a.mean())
  predicted_stock_price2 =  ytest
  for k in range(len(ytest)):
      fpr2, tpr2, thresholds2 = roc_curve(ytest[k], predicted_stock_price2[k], pos_label=1)
      auc(fpr2, tpr2)
#       b.append(auc(fpr2, tpr2))
  b=pd.DataFrame(b)
  b=b.dropna()
  print('测试2',b.mean())
  i=i+1

测试2 Series([], dtype: float64)
测试2 Series([], dtype: float64)


KeyboardInterrupt: ignored

In [0]:

i=0
modle=[]
for train,test in kf.split(xtrain):
  a = []
  b= []
  X_train, X_valid= xtrain[train, :], xtrain[test, :]
  y_train, y_valid = ytrain[train], ytrain[test]
  print(i, 'fold')
  for j in range(len(y_valid)):
      fpr1, tpr1, thresholds1 = roc_curve(y_valid[j], y_valid[j], pos_label=1)
      a.append(auc(fpr1, tpr1))
  a=pd.DataFrame(a)
  a=a.dropna()
  print("测试1:"+a.mean())
  predicted_stock_price2 =  modle[i].predict(xtest)
  for k in range(len(ytest)):
      fpr2, tpr2, thresholds2 = roc_curve(ytest[k], predicted_stock_price2[k], pos_label=1)
      auc(fpr2, tpr2)
      b.append(auc(fpr2, tpr2))
  b=pd.DataFrame(b)
  b=b.dropna()
  print("测试2:"+b.mean())
  i=i+1
# predicted_stock_price1 = regressor.predict(X_valid)
# a = []
# for j in range(len(y_valid)):
#       fpr1, tpr1, thresholds1 = roc_curve(y_valid[j], predicted_stock_price1[j], pos_label=1)
#       a.append(auc(fpr1, tpr1))
# a=pd.DataFrame(a)
# a=a.dropna()
# print("测试1:"+a.mean())
# predicted_stock_price2 = regressor.predict(xtest)
# b = []
# for k in range(len(ytest)):
#       fpr2, tpr2, thresholds2 = roc_curve(ytest[k], predicted_stock_price2[k], pos_label=1)
#       b.append(auc(fpr2, tpr2))
# b=pd.DataFrame(b)
# b=b.dropna()
# b.mean()


TypeError: ignored

In [0]:
# predicted_stock_price = regressor.predict(xtest)
# plt.figure(figsize=(30,10))
# plt.plot(ytest, color = 'red', label = 'Real Price')
# plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted')
# plt.title('Price Prediction')
# plt.xlabel('Time')
# plt.ylabel('涨跌')
# plt.legend()
# plt.show()

In [69]:
from sklearn.metrics import auc
from sklearn.metrics import roc_curve



0    0.85178
dtype: float64

# 新闻

In [0]:
df1 = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df1 = pd.concat([df1,df_add,df_test])

In [84]:
df2=df1[['name','ts_code']]
df2 = df2.drop_duplicates(keep='first')


,0,1152,2304,3456,3656,4608,5760,6912,8064,9216,10368,11520,12672,13824,14976,16128,17280,17480,18432,19584,20736,21888,23040,24192,25344,26496,27648,28800,29952,31104,32256,33408,34560,35712,36864,38016
name,农林牧渔,采掘,化工,黑色金属,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,轻工制造,医药生物,公用事业,交通运输,房地产,商业贸易,餐饮旅游,休闲服务,综合,申万制造,申万消费,申万投资,申万服务,申万300指数,建筑材料,建筑装饰,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
ts_code,801010,801020,801030,801040,801040,801050,801080,801110,801120,801130,801140,801150,801160,801170,801180,801200,801210,801210,801230,801250,801260,801270,801280,801300,801710,801720,801730,801740,801750,801760,801770,801780,801790,801880,801890,802600


In [86]:
df_news1 = pd.read_csv(r'./TRAINSET_NEWS.csv')
df_news2= pd.read_csv(r'./20190506_NEWS.csv')
df_news3= pd.read_csv(r'./20190514_NEWS.csv')
df_news = pd.concat([df_news1,df_news2,df_news3])
df_news.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27077 entries, 0 to 112
Data columns (total 4 columns):
id         27077 non-null object
date       27077 non-null int64
title      27048 non-null object
content    26915 non-null object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


In [0]:
df  

# Tensorflow

In [0]:
import tensorflow as tf
time_step=10      #时间步
rnn_unit=25       #hidden layer units
batch_size=80     #每一批次训练多少个样例
input_size=3      #输入层维度
output_size=1     #输出层维度
lr=0.0006 


In [0]:
def get_train_data(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    batch_index=[]
    data_train=dataset[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集x和y初定义
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
           batch_index.append(i)
        x=normalized_train_data[i:i+time_step,:3]
        y=normalized_train_data[i:i+time_step,2,np.newaxis]
        train_x.append(x.tolist())
        train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y
 


In [0]:
def get_test_data(time_step=10,test_begin=30000):
    data_test=dataset[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]  
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:3]
        y=normalized_test_data[i*time_step:(i+1)*time_step,2]
        test_x.append(x.tolist())
        test_y.extend(y)
    test_x.append((normalized_test_data[(i+1)*time_step:,:3]).tolist())
    test_y.extend((normalized_test_data[(i+1)*time_step:,2]).tolist())
    return mean,std,test_x,test_y

In [0]:
weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}
def lstm(X):     
    batch_size=tf.shape(X)[0]
    time_step=tf.shape(X)[1]
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [0]:
def train_lstm(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_=lstm(X)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, module_file)
        #重复训练2000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))

In [0]:
train_lstm()

ValueError: ignored